## 주간 부동산 뉴스 크롤러
- 국토부, LH, 서울시 부동산 관련 테마 보도자료 클롤링
- 실행시킨 날 포함 -5일까지 자료 가져옴 (주간 자료 만들기 위함)
- 현재는 URL만 가져옴 (본문 내용도 가져올 수 있도록 추가개발 필요)

---

크롤러_V1

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 30 15:49:32 2019

@author: JW_Mudfish
"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
import numpy as np
import datetime
import os

ua = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}


# 국토교통부
result = []
def gukto_news():
    
    #gukto_url = 'http://www.molit.go.kr/USR/NEWS/m_71/lst.jsp?search_section=p_sec_2&search=&search_gubun=1&search_gubun1=all&srch_usr_titl=Y&srch_usr_ctnt=&psize=10&search_regdate_s=2018-05-30&search_regdate_e=2019-05-30&srch_cate=&srch_dept_nm=&search_kind=&search_gbn=&lst_gbn=T&lcmspage=1'
    gukto_url = 'http://www.molit.go.kr/USR/NEWS/m_71/lst.jsp?search_section=p_sec_2&lcmspage=1&psize=20'
       
    response = requests.get(gukto_url, headers = ua)
    html = BeautifulSoup(response.text, 'html.parser')
    
    li_list = html.select("#board-list tr")
               
    for i in range(1,len(li_list)):
        ns = '국토부'                      
        title = li_list[i].select_one('a').text.replace('\n','').replace('\t','')
        url = 'http://www.molit.go.kr/USR/NEWS/m_71/' + li_list[i].select_one('a')['href']
        time = li_list[i].select('td')[-2].text
    
        result.append(['국토부',title,url,time])
        #gt = pd.DataFrame(gukto)
    return result

# LH
def lh_news():
    
    for num in range(1,4):
        lh_url = 'http://www.lh.or.kr/lh_offer/infor/inf1200_list.asp?page={}'.format(num)
        
        
        response = requests.get(lh_url, headers = ua)
        # 글자 깨져서 옵션추가
        html = BeautifulSoup(response.content.decode('euc_kr','replace'), 'html.parser')
        
        li_list = html.select("tbody tr")
        
        for i in li_list:
            title = i.select_one('a').text
            url = 'http://www.lh.or.kr/lh_offer/infor/' + i.select_one('a')['href']
            time = i.select('td')[-2].text
            
            result.append(['LH',title,url,time])
    return result


# 서울시
def seoul_news():
    seoul_url = 'http://seoulboard.seoul.go.kr/front/bbs.json?bbsNo=158&curPage=1&srchBeginDt=&srchEndDt=&srchCtgry=465&cntPerPage=20&srchKey=&srchText='
    
    response = requests.get(seoul_url, headers = ua)
    
    data = json.loads(response.text)
                          
    for i in data['listVO']['listOutptObject']:
        title = i['mapShow']['sj']
        url = 'http://spp.seoul.go.kr/main/news/news_report.jsp#view/' + str(i['mapHide']['nttNo'])
        time = i['mapShow']['writngDe'].split()[0].replace('.','-')
    
        result.append(['서울시',title,url,time])



############# 실행단 ######
result = []

gukto_news()
lh_news()
seoul_news()

######### 전처리 ##########
final_result = pd.DataFrame(result, columns = ['organization','title','url','date'])

# 오늘날짜 구하기
datetime.datetime.today().strftime('%Y-%m-%d')

tt = []
for i in range(0,5):
    aa = datetime.datetime.today() - datetime.timedelta(days=i)
    aa = aa.strftime('%Y-%m-%d')
    tt.append(aa)

#df = final_result[final_result['date'].apply(lambda x: x in tt)]
df = final_result[final_result['date'].isin(tt)]
df

,organization,title,url,date
0,국토부,"층간소음, 시공단계부터 철저히 점검한다",http://www.molit.go.kr/USR/NEWS/m_71/dtl.jsp?l...,2019-07-28
1,국토부,"청년·신혼부부 등 매입임대주택 3,942호…30일부터 입주자 모집",http://www.molit.go.kr/USR/NEWS/m_71/dtl.jsp?l...,2019-07-28
